# MLP Predictor for Perovskite Solar Cells
Predicts PCE, Voc, Jsc, and FF using an MLP neural network with TensorFlow/Keras.
Dataset: `Final Dataset.csv`


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv('Final Dataset.csv')
df.dropna(inplace=True)
df.head()

In [ ]:
target_columns = ['PCE', 'Voc', 'Jsc', 'FF']
X = df.drop(columns=target_columns)
y = df[target_columns]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Normalize input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(4)  # Predicting PCE, Voc, Jsc, FF
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

In [ ]:
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.1,
    epochs=100,
    batch_size=16,
    verbose=1
)

In [ ]:
y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Test MSE: {mse:.4f}')
print(f'Test R² Score: {r2:.4f}')

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.title('Training History')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model.save('perovskite_mlp_model')
print('Model saved to perovskite_mlp_model/')